In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandat.csv") 
#on récupère le csv de la liste des mandats

In [ ]:
mandat_dep=df.sort_values(by='Département')


In [ ]:
mandat_dep['Département'].unique()

In [ ]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements mentionnés
dep_possible.remove(" Département d'élection inconnu")
for dep in dep_possible :#on enlève les nan
    if type(dep)!=str :
        dep_possible.remove(dep)
#dep_possible
#attention, il y a souvent un espace avant !
print(dep_possible)

Voilà une autre possiblité

In [ ]:
dep_possible=list(mandat_dep['Département'].unique())[:-1]#une liste de tous les départements mentionnés
dep_possible.remove(" Département d'élection inconnu")
dep_possible

La suite

In [ ]:
mandat_dep.count()#on vérifie qu'il n'y a pas trop de manques

In [ ]:
mandat_dep.set_index('Département', inplace=True)#on met département en index pour pouvoir scinder par départements

In [ ]:
liste_dep={}#on crée un dictionnaire où on va mettre les dataframes par départements
for dep in dep_possible:#pour chaque département
    mand_du_dep=mandat_dep.loc[dep]#on récupère le dataframe des mandats dans le département  
    liste_dep[dep]=mand_du_dep#et on l'ajoute au dictionnaire

In [ ]:
carac_dep={}
for dep in dep_possible :
    nbre_mandat=liste_dep[dep]['nom du député'].count()#on compte le nombre total de mandat dans le département
    cpte_depute=0#on crée la variable qui va compter le nombre de députés différents
    compte=[]#on crée une liste où on va mettre les députés déjà comptés
    for depute in liste_dep[dep]['nom du député'] :#on boucle sur les noms de députés
        if depute not in compte :#si on ne l'a pas déjà compté
            cpte_depute+=1#on rajoute un au compte
            compte.append(depute)#et on indique qu'on l'a désormais compté
    carac_dep[dep]=pd.DataFrame(['nombre_mandat', 'nombre député','nombre moyen de mandats par député'], [nbre_mandat, cpte_depute, nbre_mandat/cpte_depute])#on crée un dataframe avec les informations
    
carac_dep


Voilà un code plus court mais que je crois équivalent à ce que tu as fait :

In [ ]:
mandat_dep.reset_index()
x,y,z=[],[],[]
for dep in dep_possible :
    x+=[liste_dep[dep]['nom du député'].count()]#on compte le nombre total de mandat dans le département
    y+=[len(df[df['Département']==dep]['nom du député'].unique())]#Puis le nombre de députés différent par dpt
    z+=[x[-1]/y[-1]] #Ceci est le nombre moyen de mandat par député
    
carac_dep_bis=np.array([x, y, z])#on crée un dataframe avec les informations
df = pd.DataFrame(carac_dep_bis, index=['nombre_mandat', 'nombre député','nombre moyen de mandats par député'],columns=dep_possible)
df.head()

#### Analyse des NAN

Les nan de département sont très majoitairement issu de la révolution :

In [93]:
df[df['Département'].isna()].count()

Unnamed: 0          1330
Département            0
Groupe              1302
Législature         1317
Mandat              1317
Régime politique    1314
nom du député       1330
dtype: int64

In [95]:
df[df['Département'].isna()][df['Régime politique']!='Révolution'].count()

<ipython-input-95-54b5895b7c85>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['Département'].isna()][df['Régime politique']!='Révolution'].count()


Unnamed: 0          30
Département          0
Groupe               6
Législature         17
Mandat              17
Régime politique    14
nom du député       30
dtype: int64

Pour les régime politique, on a les cas super-pathologique + des personnes de cette législature un peu perdu

In [98]:
df[df['Régime politique'].isna()]

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,nom du député
1050,0,NaN,2,(Législature en cours),Depuis le 26 octobre 2020 (mandat en cours),NaN,François Baroin
24363,0,NaN,NaN,NaN,NaN,NaN,"Charles, André, Marie, Joseph de Gaulle"
24379,0,NaN,NaN,NaN,NaN,NaN,"Jean, Joseph, Paul, Augustin Dessolle"
24412,0,NaN,NaN,(Législature en cours),Depuis le 26 octobre 2020 (mandat en cours),NaN,"Joseph, François, Marie Kermorial"
28099,0,NaN,NaN,(Législature en cours),Depuis le 26 octobre 2020 (mandat en cours),NaN,"Emmanuel, Joseph Sieyès"
32530,0,NaN,NaN,NaN,NaN,NaN,Alexandre Bachelet
32531,0,NaN,NaN,NaN,NaN,NaN,Henry Senes
32532,0,NaN,NaN,NaN,NaN,NaN,Camille Rolland
32533,0,NaN,NaN,NaN,NaN,NaN,Jean-Pierre Rambaut
32534,0,NaN,NaN,NaN,NaN,NaN,Georges Pezières


La ce n'est que les cas super pathologiques 

In [99]:
df[df['Mandat'].isna()]

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,nom du député
24363,0,NaN,NaN,NaN,NaN,NaN,"Charles, André, Marie, Joseph de Gaulle"
24379,0,NaN,NaN,NaN,NaN,NaN,"Jean, Joseph, Paul, Augustin Dessolle"
32530,0,NaN,NaN,NaN,NaN,NaN,Alexandre Bachelet
32531,0,NaN,NaN,NaN,NaN,NaN,Henry Senes
32532,0,NaN,NaN,NaN,NaN,NaN,Camille Rolland
32533,0,NaN,NaN,NaN,NaN,NaN,Jean-Pierre Rambaut
32534,0,NaN,NaN,NaN,NaN,NaN,Georges Pezières
32535,0,NaN,NaN,NaN,NaN,NaN,Victor Le Gorgeu
32536,0,NaN,NaN,NaN,NaN,NaN,François Labrousse
32537,0,NaN,NaN,NaN,NaN,NaN,Paul Fleurot


In [100]:
df[df['Groupe'].isna()]

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,nom du député
31,0,Finistère,NaN,XIIIe législature,Du 17 juin 2012 au 19 juin 2012,Cinquième République - Assemblée nationale,Yvon Abiven
41,0,Tarn,NaN,IIIe législature,Du 23 mars 1884 au 14 octobre 1885,Troisième République - Chambre des députés,"Jean, Pierre, Léon Abrial"
42,0,Tarn,NaN,Ve législature,Du 22 septembre 1889 au 14 octobre 1893,Troisième République - Chambre des députés,"Jean, Pierre, Léon Abrial"
43,0,Tarn,NaN,VIe législature,Du 3 septembre 1893 au 4 février 1894,Troisième République - Chambre des députés,"Jean, Pierre, Léon Abrial"
98,0,Sarthe,NaN,VIe législature,Du 11 avril 1897 au 31 mai 1898,Troisième République - Chambre des députés,François d'Aillières
...,...,...,...,...,...,...,...
37756,0,Seine-Inférieure,NaN,Conseil des Cinq-Cents,Du 11 avril 1797 au 4 septembre 1797,Révolution,François Grégoire de Rumare
37757,0,Moselle,NaN,,Du 8 février 1871 au 1er mars 1871,Assemblée Nationale,"Pierre, Paul Bardon"
37758,0,Sénégal,NaN,Assemblée nationale législative,Du 17 août 1851 au 14 novembre 1851,Deuxième République,John Sleigth
37759,0,Marne,NaN,Corps législatif,Du 30 novembre 1803 au 31 décembre 1808,Consulat et Premier Empire,François Becquey
